In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import healpy as hp
from matplotlib.ticker import FormatStrFormatter

In [ ]:
font = {'size'   : 20, 'family':'STIXGeneral'}
plt.rcParams.update({
     'text.usetex': True,
     'font.family': 'serif',
     'font.sans-serif': ['Computer Modern'],
     'font.size':20})
plt.rc_context({'axes.autolimit_mode': 'round_numbers'})

### To run this notebook, first run all the pipelines with each of the example yaml files, using the same output_dir for all of them. 

In [ ]:
save = True #whether to save plots from this notebook
output_dir = '/Users/kristen/Documents/GitHub/NILC-Inference-Pipeline/plotting_notebooks/tests/outputs_90_150' #output_dir that was used in the yaml files
ellmax = 250 #maximum ell used in yaml
Nbins = 10 #number of bins used in yaml
tsz_type = 'nongaussian' #'gaussian' or 'nongaussian'

In [ ]:
# Load Files
data_vec_dir = output_dir + '/data_vecs'
Clij_mult = pickle.load(open(f'{data_vec_dir}/Clij_multifrequency_{tsz_type}tsz_2ksims_tszamp150_gaussianlkl.p', 'rb'))
Clpq_hilc_wts_vary = pickle.load(open(f'{data_vec_dir}/Clpq_hilc_{tsz_type}tsz_weightsvary_2ksims_tszamp150_gaussianlkl_sr.p', 'rb'))
Clpq_hilc_wts_once = pickle.load(open(f'{data_vec_dir}/Clpq_hilc_{tsz_type}tsz_weightsonce_2ksims_tszamp150_gaussianlkl_sr.p', 'rb'))
Clpq_nilc = pickle.load(open(f'{data_vec_dir}/Clpq_nilc_{tsz_type}tsz_2ksims_tszamp150_gaussianlkl_4scales.p', 'rb'))


# Mean input power spectra over all simulations

In [ ]:
def tsz_spectral_response(freqs):
    '''
    ARGUMENTS
    ---------
    freqs: 1D numpy array, contains frequencies (GHz) for which to calculate tSZ spectral response

    RETURNS
    ---------
    1D array containing tSZ spectral response to each frequency
    '''
    T_cmb = 2.726
    h = 6.62607004*10**(-34)
    kb = 1.38064852*10**(-23)
    response = []
    for freq in freqs:
        x = h*(freq*10**9)/(kb*T_cmb) #x is v/56.9 GHz
        response.append(T_cmb*(x*1/np.tanh(x/2)-4))
    return np.array(response)
    

In [ ]:
g90, g150 = tsz_spectral_response([90,150])

In [ ]:

Clij_mult_mean = np.mean(Clij_mult, axis=0)
cmb_fiducial = Clij_mult_mean[0,0,1]
tsz_90_fiducial = Clij_mult_mean[0,0,2]
tsz_150_fiducial = Clij_mult_mean[1,1,2]
tsz_90x150_fiducial = Clij_mult_mean[0,1,2]
total_90_fiducial = Clij_mult_mean[0,0,0]
total_150_fiducial = Clij_mult_mean[1,1,0]
total_90x150_fiducial = Clij_mult_mean[0,1,0]

ells = np.arange(ellmax+1)
res = stats.binned_statistic(ells[2:], ells[2:], statistic='mean', bins=Nbins)
mean_ells = np.round((res[1][:-1]+res[1][1:])/2)
to_dl = mean_ells*(mean_ells+1)/2/np.pi

plt.plot(mean_ells, to_dl*cmb_fiducial, label='CMB', linestyle='dashed', color='k')
plt.plot(mean_ells, to_dl*tsz_90_fiducial, label='ftSZ 90 GHz', linestyle='dotted', color='blue')
plt.plot(mean_ells, to_dl*tsz_150_fiducial, label='ftSZ 150 GHz', linestyle='dotted', color='red')
plt.plot(mean_ells, to_dl*tsz_90x150_fiducial, label='ftSZ 90x150 GHz', linestyle='dotted', color='green')

plt.plot(mean_ells, to_dl*total_90_fiducial, label='Total 90 GHz', color='blue')
plt.plot(mean_ells, to_dl*total_150_fiducial, label='Total 150 GHz', color='red')
plt.plot(mean_ells, to_dl*total_90x150_fiducial, label='Total 90x150 GHz', color='green')

plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$')
plt.legend(fontsize=12)
plt.grid()
plt.xlim(mean_ells[0], mean_ells[-1])



# Input power spectra for one simulation

In [ ]:
sim=0
cmb= Clij_mult[sim,0,0,1]
tsz_90 = Clij_mult[sim,0,0,2]
tsz_150 = Clij_mult[sim,1,1,2]
tsz_90x150 = Clij_mult[sim,0,1,2]
total_90 = Clij_mult[sim,0,0,0]
total_150 = Clij_mult[sim,1,1,0]
total_90x150 = Clij_mult[sim,0,1,0]

ells = np.arange(ellmax+1)
res = stats.binned_statistic(ells[2:], ells[2:], statistic='mean', bins=Nbins)
mean_ells = np.round((res[1][:-1]+res[1][1:])/2)
to_dl = mean_ells*(mean_ells+1)/2/np.pi

plt.plot(mean_ells, to_dl*cmb, label='CMB', linestyle='dashdot', color='k')
plt.plot(mean_ells, to_dl*tsz_90, label='ftSZ 90 GHz', linestyle='dotted', color='blue')
plt.plot(mean_ells, to_dl*tsz_150, label='ftSZ 150 GHz', linestyle='dotted', color='red')
plt.plot(mean_ells, to_dl*tsz_90x150, label=r'ftSZ 90$\times$150 GHz', linestyle='dotted', color='green')

plt.plot(mean_ells, to_dl*(cmb_fiducial+tsz_90), label='CMB+ftSZ 90 GHz', linestyle='solid', color='blue')
plt.plot(mean_ells, to_dl*(cmb_fiducial+tsz_150), label='CMB+ftSZ 150 GHz', linestyle='solid', color='red')
plt.plot(mean_ells, to_dl*(cmb_fiducial+tsz_90x150), label=r'CMB+ftSZ 90$\times$150 GHz', linestyle='solid', color='green')

plt.plot(mean_ells, to_dl*total_90, label='Total 90 GHz', color='cyan', linestyle='dashed')
plt.plot(mean_ells, to_dl*total_150, label='Total 150 GHz', color='orange', linestyle='dashed')
plt.plot(mean_ells, to_dl*total_90x150, label=r'Total 90$\times$150 GHz', color='lime', linestyle='dashed')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$ $[\mathrm{K}^2]$')
plt.legend(fontsize=11)
plt.grid()
plt.xlim(mean_ells[0], mean_ells[-1])
if save:
    plt.savefig('input_spectra_one_sim.pdf', bbox_inches='tight')

# Mean ILC spectra over all simulations

In [ ]:
def get_ILC_spectra(Clpq):
    '''
    ARGUMENTS
    ---------
    Clpq: (Nscalings, 2, 2, N_preserved_comps, N_preserved_comps, Nbins) ndarray containing
        scaled and unscaled ILC map auto- and cross-spectra

    RETURNS
    ---------
    ClTT: (Nbins, ) array containing auto-spectrum of T map
    ClTy: (Nbins, ) array containing cross-spectrum of T and y map
    Clyy: (Nbins, ) array containing auto-spectrum of y map
    '''
    Clpq = Clpq[0,0,0]
    ClTT, ClTy, Clyy = Clpq[0,0], Clpq[0,1], Clpq[1,1]
    return ClTT, ClTy, Clyy

In [ ]:
Clij_mult_mean = np.mean(Clij_mult, axis=0)
cmb_fiducial = Clij_mult_mean[0,0,1]
comptony_fiducial = Clij_mult_mean[0,0,2]/g90**2

ells = np.arange(ellmax+1)
res = stats.binned_statistic(ells[2:], ells[2:], statistic='mean', bins=Nbins)
mean_ells = np.round((res[1][:-1]+res[1][1:])/2)
to_dl = mean_ells*(mean_ells+1)/2/np.pi

cltt_hilc, clty_hilc, clyy_hilc = get_ILC_spectra(np.mean(Clpq_hilc_wts_once, axis=0))
cltt_nilc, clty_nilc, clyy_nilc = get_ILC_spectra(np.mean(Clpq_nilc, axis=0))

plt.plot(mean_ells, to_dl*cmb_fiducial, label='CMB', linestyle='solid', color='blue')
plt.plot(mean_ells, to_dl*np.zeros_like(cmb_fiducial), label='CMB ' + r'$\times$' + '\n' + r'Amplified Compton-$y$', linestyle='solid', color='red')
plt.plot(mean_ells, to_dl*comptony_fiducial, label=r'Amplified Compton-$y$', linestyle='solid', color='darkgreen')
plt.plot(mean_ells, to_dl*cltt_hilc, label=r'$C_\ell^{\hat{T}\hat{T}}$'+ '\nharmonic ILC', linestyle='dashed', color='lightblue')
plt.plot(mean_ells, to_dl*clty_hilc, label=r'$C_\ell^{\hat{T}\hat{y}}$'+ '\nharmonic ILC', linestyle='dashed', color='orange')
plt.plot(mean_ells, to_dl*clyy_hilc, label=r'$C_\ell^{\hat{y}\hat{y}}$'+ '\nharmonic ILC', linestyle='dashed', color='mediumspringgreen')
plt.plot(mean_ells, to_dl*cltt_nilc, label=r'$C_\ell^{\hat{T}\hat{T}}$'+ '\nneedlet ILC', linestyle='dotted', color='cyan')
plt.plot(mean_ells, to_dl*clty_nilc, label=r'$C_\ell^{\hat{T}\hat{y}}$'+ '\nneedlet ILC', linestyle='dotted', color='salmon')
plt.plot(mean_ells, to_dl*clyy_nilc, label=r'$C_\ell^{\hat{y}\hat{y}}$'+ '\nneedlet ILC', linestyle='dotted', color='olive')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$')
plt.legend(fontsize=12, ncol=1, bbox_to_anchor=(1.0, 1.0))
plt.grid()
plt.xlim(mean_ells[0], mean_ells[-1])
if save:
    plt.savefig('ILC_spectra.pdf', bbox_inches='tight')

# ILC spectra for a few simulations

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10,5), sharey=True)

sims = [0,1,2]
for i in range(3):
    plt.axes(axs[i])
    
    sim = sims[i]

    cmb = Clij_mult[sim, 0,0,1]
    comptony = Clij_mult[sim, 0,0,2]/g90**2

    ells = np.arange(ellmax+1)
    res = stats.binned_statistic(ells[2:], ells[2:], statistic='mean', bins=Nbins)
    mean_ells = np.round((res[1][:-1]+res[1][1:])/2)
    to_dl = mean_ells*(mean_ells+1)/2/np.pi

    cltt_hilc, clty_hilc, clyy_hilc = get_ILC_spectra(Clpq_hilc_wts_vary[sim])
    cltt_nilc, clty_nilc, clyy_nilc = get_ILC_spectra(Clpq_nilc[sim])

    plt.plot(mean_ells, to_dl*cmb_fiducial, label='CMB', linestyle='solid', color='blue')
    plt.plot(mean_ells, to_dl*np.zeros_like(cmb_fiducial), label='CMB x Compton-y', linestyle='solid', color='red')
    plt.plot(mean_ells, to_dl*comptony_fiducial, label='Compton-y', linestyle='solid', color='darkgreen')
    plt.plot(mean_ells, to_dl*cltt_hilc, label='ClTT HILC', linestyle='dashed', color='lightblue')
    plt.plot(mean_ells, to_dl*clty_hilc, label='ClTy HILC', linestyle='dashed', color='orange')
    plt.plot(mean_ells, to_dl*clyy_hilc, label='Clyy HILC', linestyle='dashed', color='mediumspringgreen')
    plt.plot(mean_ells, to_dl*cltt_nilc, label='ClTT NILC', linestyle='dotted', color='cyan')
    plt.plot(mean_ells, to_dl*clty_nilc, label='ClTy NILC', linestyle='dotted', color='salmon')
    plt.plot(mean_ells, to_dl*clyy_nilc, label='Clyy NILC', linestyle='dotted', color='olive')
    plt.xlabel(r'$\ell$')
    if i==0:
        plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$')
    plt.grid()
    plt.xlim(mean_ells[0], mean_ells[-1])
    plt.title(f'Simulation {sim}', fontsize=16)
    
handles, labels = axs[-1].get_legend_handles_labels() 
fig.legend(handles, labels, fontsize=12, bbox_to_anchor=(0.85, 0.0), ncol=3)

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(10,5), sharey=True)

sims = [0,1,2]
for i in range(3):
    plt.axes(axs[i])
    
    sim = sims[i]

    cmb = Clij_mult[sim, 0,0,1]
    comptony = Clij_mult[sim, 0,0,2]/g90**2

    ells = np.arange(ellmax+1)
    res = stats.binned_statistic(ells[2:], ells[2:], statistic='mean', bins=Nbins)
    mean_ells = np.round((res[1][:-1]+res[1][1:])/2)
    to_dl = mean_ells*(mean_ells+1)/2/np.pi

    cltt_hilc, clty_hilc, clyy_hilc = get_ILC_spectra(Clpq_hilc_wts_vary[sim])
    cltt_nilc, clty_nilc, clyy_nilc = get_ILC_spectra(Clpq_nilc[sim])

    plt.plot(mean_ells, to_dl*cmb_fiducial, label='CMB', linestyle='solid', color='blue')
    plt.plot(mean_ells, to_dl*np.zeros_like(cmb_fiducial), label='CMB x Compton-y', linestyle='solid', color='red')
    plt.plot(mean_ells, to_dl*comptony_fiducial, label='Compton-y', linestyle='solid', color='darkgreen')
    plt.plot(mean_ells, to_dl*cltt_hilc, label='ClTT HILC', linestyle='dashed', color='lightblue')
    plt.plot(mean_ells, to_dl*clty_hilc, label='ClTy HILC', linestyle='dashed', color='orange')
    plt.plot(mean_ells, to_dl*clyy_hilc, label='Clyy HILC', linestyle='dashed', color='mediumspringgreen')
    plt.plot(mean_ells, to_dl*cltt_nilc, label='ClTT NILC', linestyle='dotted', color='cyan')
    plt.plot(mean_ells, to_dl*clty_nilc, label='ClTy NILC', linestyle='dotted', color='salmon')
    plt.plot(mean_ells, to_dl*clyy_nilc, label='Clyy NILC', linestyle='dotted', color='olive')
    plt.xlabel(r'$\ell$')
    if i==0:
        plt.ylabel(r'$\ell(\ell+1)C_\ell/(2\pi)$')
    plt.grid()
    plt.xlim(mean_ells[0], mean_ells[-1])
    plt.title(f'Simulation {sim}', fontsize=16)
    
handles, labels = axs[-1].get_legend_handles_labels() 
fig.legend(handles, labels, fontsize=12, bbox_to_anchor=(0.85, 0.0), ncol=3)

# Plot Covariance and Correlation Matrices

In [ ]:
from matplotlib.colors import SymLogNorm, LogNorm

In [ ]:
def get_PScov_sim(Clpq_unscaled):
    '''
    ARGUMENTS
    ---------
    inp: Info object containing input paramter specifications
    Clpq_unscaled: (Nsims, N_preserved_comps=2, N_preserved_comps=2, Nbins) ndarray 
        containing propagation of each pair of component maps
        to NILC map auto- and cross-spectra
    
    RETURNS
    -------
    cov: (3*Nbins, 3*Nbins) ndarray containing covariance matrix Cov_{pq,rs}
        index as cov[(0-2 for ClTT ClTy Clyy)*Nbins + bin1, (0-2 for ClTT ClTy Clyy)*Nbins + bin2]
    '''
    Clpq_tmp = Clpq_unscaled
    Clpq_tmp = np.array([Clpq_tmp[:,0,0], Clpq_tmp[:,0,1], Clpq_tmp[:,1,1]])
    Clpq_tmp = np.transpose(Clpq_tmp, axes=(0,2,1)) #shape (3 for ClTT, ClTy, Clyy, Nbins, Nsims)
    Clpq_tmp = np.reshape(Clpq_tmp, (Nbins*3, -1))
    cov = np.cov(Clpq_tmp)
    return cov

In [ ]:
def plot_cov_matrix(Clpq, title=None, corr=False, colorbar=False):
    PScov = get_PScov_sim(Clpq[:,0,0,0])
    if not corr:
        plt.imshow(np.abs(PScov), norm=LogNorm(vmin=1.e-28, vmax=1.e-24))
    else:
        diag = np.diag(PScov)
        PScorr = np.einsum('ab,a,b->ab', PScov, 1/np.sqrt(diag), 1/np.sqrt(diag))
        plt.imshow(PScorr, vmin=-1., vmax=1.)
    ax = plt.gca()
    ax.set_xticks([5,15,25])
    ax.set_yticks([5,15,25])
    ax.set_xticklabels([r'$C_\ell^{\hat{T}\hat{T}}$', r'$C_\ell^{\hat{T}\hat{y}}$', r'$C_\ell^{\hat{y}\hat{y}}$'])
    ax.set_yticklabels([r'$C_\ell^{\hat{T}\hat{T}}$', r'$C_\ell^{\hat{T}\hat{y}}$', r'$C_\ell^{\hat{y}\hat{y}}$'])
    if colorbar:
        plt.colorbar()
    if title:
        plt.title(title, fontsize=15)

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(9,6), sharey=True, sharex=True, layout='constrained')
axs = axs.flatten()
ax = 0
for i in range(2):
    corr = (not i)
    corr_str = 'Corr' if corr else (r'$|$Cov$|$')
    for j in range(3):
        plt.axes(axs[ax])
        title = corr_str
        if j==0:
            Clpq = Clpq_nilc
            title = 'NILC ' + title
        elif j==1:
            Clpq = Clpq_hilc_wts_vary
            title = 'HILC (weights vary) ' + title
        else:
            Clpq = Clpq_hilc_wts_once
            title = 'HILC (weights fixed) ' + title
        colorbar=True if j==2 else False
        plot_cov_matrix(Clpq, title=title, corr=corr, colorbar=colorbar)
        ax += 1
if save:
    plt.savefig('cov.pdf', bbox_inches='tight')
        
        
    